<a href="https://colab.research.google.com/github/Mounish2023/Build-LLMFromScratch/blob/main/gptModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install datasets tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platfo

# **preprocessing text**

In [ ]:

import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding(encoding_name='gpt2')

token_ids = tokenizer.encode(text=raw_text, allowed_special={'<|endoftext|>'})

In [ ]:
len(token_ids)

5145

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TDatasetV1(Dataset):

  def __init__(self, txt, tokenizer, max_length,stride) -> None:

    super().__init__()
    self.input_ids = []
    self.output_ids = []
    tokenids = tokenizer.encode(txt)

    for i in range(0,len(tokenids)-max_length, stride):
      input_chunk = tokenids[i:i+max_length]
      output_chunk = tokenids[i+1:i+1+max_length]

      self.input_ids.append(torch.tensor(input_chunk))
      self.output_ids.append(torch.tensor(output_chunk))

    def __len__(self):
      return len(self.input_ids)
    def __getitem__(self,idx):
      return self.input_ids[idx], self.target_ids[idx]



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids)- max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1: i+1+max_length]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]






In [ ]:
def create_dataloader_v1(raw_text, batch_size=4, max_length=256, stride=128, num_workers=0, shuffle=True, drop_last=True):
  tokenizer = tiktoken.get_encoding('gpt2')
  dataset = GPTDatasetV1(txt=raw_text, tokenizer=tokenizer, max_length=max_length, stride=stride)

  dataloader = DataLoader(
      dataset=dataset,
      batch_size=batch_size,
      drop_last=drop_last,
      shuffle=shuffle,
      num_workers=num_workers)

  return dataloader



In [ ]:
dataloader = create_dataloader_v1(raw_text=raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
dataloader_iterator = iter(dataloader)
first_batch = next(dataloader_iterator)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(dataloader_iterator)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
dataloader = create_dataloader_v1(raw_text=raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
dataloader_iterator = iter(dataloader)
inputs, targets = next(dataloader_iterator)
print(f'inputs - {inputs}')
print(f'targets - {targets}')

inputs - tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
targets - tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [ ]:
import torch
torch.__version__

'2.6.0+cu124'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
vocab_size = tokenizer.n_vocab
output_dim = 256


token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
positional_embeddings_layer = torch.nn.Embedding(vocab_size,output_dim)
max_length = 4
positional_embeddings = positional_embeddings_layer(torch.arange(0,max_length))


In [ ]:
positional_embeddings.shape

torch.Size([4, 256])

In [ ]:
input_embeddings = token_embeddings + positional_embeddings

In [ ]:
print(f"token embeddings 1st input in a batch of 8, 1 token of 4 tokens - {token_embeddings[0][0]}")
print(f"positional embeddings  - {positional_embeddings[0]}")


print(f"input embeddings {input_embeddings[0][0]}")

token embeddings 1st input in a batch of 8, 1 token of 4 tokens - tensor([ 1.0296,  1.0401, -0.6771, -0.1172, -0.1241, -0.6753,  0.7377, -0.8750,
         0.5776,  0.6321, -2.4552, -1.0972, -0.5282, -2.5859, -1.2853, -2.0105,
        -0.0336, -0.5712,  0.9947,  0.0034, -0.3225, -0.4768,  0.0820, -0.1074,
        -0.8671,  0.0381, -0.6371,  0.8613,  0.6639, -0.7763,  0.4961, -1.6497,
         1.6120,  1.2321, -1.3022, -1.7662,  1.3133, -1.0494,  0.7037, -0.4601,
        -2.3843,  0.8889,  1.0387,  0.1590, -0.4012,  0.4184,  1.3030, -0.7208,
        -1.4654, -1.6223, -1.2178, -1.5378, -1.1688, -0.3031,  0.9016,  0.3586,
        -0.0516, -1.7023, -0.0263, -0.8325,  0.0276,  0.4681,  1.0670,  0.0550,
         1.6616,  0.0546, -1.0288,  1.4983,  0.3642, -0.5943,  0.7608,  0.3250,
         0.4177,  0.5092, -0.0831, -0.2091, -1.4425,  0.2881, -0.6624, -2.3052,
         0.6466,  1.2355, -0.8156,  0.4085, -0.9096, -0.0654, -1.1424,  0.7434,
        -0.0183, -0.7447,  2.8265,  0.3470,  1.1333,  

In [ ]:
input_embeddings.shape

torch.Size([8, 4, 256])

# **coding self attention**

In [ ]:
input_text_sf = "your journey starts with one step"
tokenizer_sf = tiktoken.get_encoding('gpt2')
token_ids_sf = tokenizer.encode(input_text_sf)
print(token_ids_sf)

[14108, 7002, 4940, 351, 530, 2239]


In [ ]:
token_embedding_layer_sf = torch.nn.Embedding(num_embeddings=vocab_size,embedding_dim=3 )

In [ ]:
token_embeddings_sf = token_embedding_layer_sf(torch.tensor(token_ids_sf))
token_embeddings_sf.shape

torch.Size([6, 3])

In [ ]:
max_length_sf = 6
positional_embedding_layer_sf = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=3)
positional_embeddings_sf = positional_embedding_layer_sf(torch.arange(0,max_length_sf))
positional_embeddings_sf.shape

torch.Size([6, 3])

In [ ]:
input_embeddings_sf = token_embeddings_sf + positional_embeddings_sf

In [ ]:
input_embeddings_sf.shape

torch.Size([6, 3])

In [ ]:
input_embeddings_sf


tensor([[-1.0265,  0.8756,  1.1912],
        [-1.3094, -1.4912,  1.0598],
        [ 1.4443, -0.7874, -0.7694],
        [-0.0126, -0.6046,  0.5222],
        [ 0.2599, -0.5985, -0.6705],
        [-1.1020,  0.6097,  1.0864]], grad_fn=<AddBackward0>)

In [ ]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
# USING for loops
attention_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attention_scores[i,j] = torch.dot(x_i, x_j)


In [ ]:
#instead of doing for loops. pytorch gives efficient way of multiplying these matrices (computing attention scores)

attention_scores = inputs @ inputs.T

In [ ]:
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
#now we have to normalize them so that there will be training stability
# we can do averages.
# but the most efficient way to do normalization is using softmax (our own softmax code) or pytorch softmax code -as it can handle the task more efficiently - making sure that the number overflow and other issues wont occur
attention_weights = torch.softmax(attention_scores, dim=-1)
#if we

In [ ]:
attention_weights
# if we add up the attention weights - it will give you 1

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [ ]:
attention_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [ ]:
# Now we have to calculate context vector for each token using attention weights and the input vectors.
 # (remember these attention weights are calculated using the dot product of the current input vector with all the other vectors - which in turn have positional information encodede and also the initialized token embeddings )

context_vector = attention_weights @ inputs




In [ ]:
context_vector

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])